In [ ]:
import os
os.getcwd()
import pandas as pd


In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git transformers bitsandbytes datasets
!pip install httpx

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
df = pd.read_csv('Contrastive.csv')
df.head(5)

,emotion,utterance,art_style,painting,anchor_art_style,anchor_painting,repetition
0,sadness,This painting shows a gloomy day. The darkness...,Realism,johan-hendrik-weissenbruch_windy-day,Realism,johan-hendrik-weissenbruch_ship-canal,1
1,sadness,the weather and atmosphere makes me feel sad,Romanticism,ivan-aivazovsky_landscape-with-a-schooner-1880,Realism,johan-hendrik-weissenbruch_ship-canal,45
2,sadness,"Dark and moody, cloudy sky that is depressing",Impressionism,paul-gauguin_clearing-1873,Realism,johan-hendrik-weissenbruch_ship-canal,18
3,sadness,the picture is dark with a somber tone,Impressionism,john-henry-twachtman_winter-landscape,Realism,johan-hendrik-weissenbruch_ships-in-harbour,7
4,sadness,the dark colors make the painting look gloomy,Impressionism,eugene-boudin_trouville-the-port-1864,Realism,johan-hendrik-weissenbruch_ships-in-harbour,5


In [ ]:
# data exploration
print('total unique anchor paintings: ',len(set(df['anchor_painting'])))
print('total unique paintings: ',len(set(df['painting'])))
print('total unique art styles: ',len(set(df['anchor_painting'])))

total unique anchor paintings:  49824
total unique paintings:  50905
total unique art styles:  49824


In [ ]:
# download the image files themselves (the original ZIP FILE IS 25G so we'll scrape a small subset lol)
import httpx
from bs4 import BeautifulSoup

base_url = "https://uploads8.wikiart.org/images/"
stopper = 0
for item in df.iterrows():
  item_name_spl = item[1]['painting'].split('_')
  painter = item_name_spl[0]
  painting = item_name_spl[1]
  scrape_url = base_url + painter + '/' + painting + '.jpg!Large.jpg'

  with open('imgs/'+item[1]['painting']+'.jpeg','wb') as outf:
    img = httpx.get(scrape_url)
    outf.write(img.content)

  if stopper == 25:
    break
  stopper += 1

process data

In [ ]:
print(df.iloc[0]['painting'])

johan-hendrik-weissenbruch_windy-day


In [ ]:
# define a dataset class
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

class ArtemisDataset(Dataset):
    def __init__(self, processor, labels_fname="Contrastive.csv", img_dirname="imgs/"):
        self.img_labels = pd.read_csv(labels_fname).iloc[:24]
        self.img_dirname = img_dirname
        self.processor = processor

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        painting_name = self.img_labels.iloc[idx]['painting'] + '.jpeg'
        img_path = os.path.join(self.img_dirname, painting_name)
        image = read_image(img_path)
        label = self.img_labels.iloc[idx]['utterance']

        dat = {"image":image,"text":label}
      # #  emotion = self.img_labels.iloc[idx]['emotion']

      #   ret_item = {"image":self.processor(images=image, text=label, return_tensors="pt").to("cuda",torch.float16),
      #               "label":self.tokenizer()}

      #  # encoding = self.processor(images=image, text=label, return_tensors="pt").to("cuda",torch.float16)

      #   # remove batch dimension
      #  # encoding = {k: v.squeeze() for k, v in encoding.items()}
      #  # encoding["input_ids"] = self.processor.tokenizer(label)["input_ids"]
      #  # encoding["attention_mask"] = self.processor.tokenizer(label)["attention_mask"]
      #  # encoding["emotion"] = emotion

        return dat #encoding

load the model and img processor

In [ ]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch

processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("ybelkada/blip2-opt-2.7b-fp16-sharded", device_map="auto", load_in_8bit=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

pytorch_model-00005-of-00008.bin:   0%|          | 0.00/997M [00:00<?, ?B/s]

pytorch_model-00006-of-00008.bin:   0%|          | 0.00/997M [00:00<?, ?B/s]

pytorch_model-00007-of-00008.bin:   0%|          | 0.00/997M [00:00<?, ?B/s]

pytorch_model-00008-of-00008.bin:   0%|          | 0.00/782M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj"]
)

model = get_peft_model(model, lora_config)

In [ ]:
train_dataset = ArtemisDataset(processor=processor)
#train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=1)#, collate_fn=collate_fn)

In [ ]:
print('testing: ')
img = read_image("/content/imgs/alfred-sisley_bank-in-the-autumn-1879.jpeg")
# tenc = processor(img, "this is a caption", padding="max_length", truncation=True, return_tensors="pt")
# tenc_txt = processor.tokenizer("this is a caption")

inputs = processor(images=img, text="this is a caption", return_tensors="pt").to("cuda", torch.float16)
outs = model(**inputs)
print(outs)

testing: 


In [ ]:
import torch

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
device = "cuda" if torch.cuda.is_available() else "cpu"

model.train()
for epoch in range(1):
  try: # this one is bc some images aren't jpeg (?)
    for idx, item in enumerate(train_dataset):
      inputs = processor(images=item["image"],
                         text=item["text"],
                         return_tensors="pt").to("cuda",torch.float16)
      outputs = model(**inputs)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
  except Exception as e:
    print('error msg: ',e)
    continue

loss keys:  odict_keys(['last_hidden_state', 'pooler_output'])
error msg:  'dict' object has no attribute 'backward'
loss keys:  odict_keys(['last_hidden_state', 'pooler_output'])
error msg:  'dict' object has no attribute 'backward'
